# Linear Regression for Forecasting Spreads

### Import Requirements

In [1]:
import pandas as pd
import numpy as np
import pandas_ta as ta

### Data Preparation and Feature Engineering

In [ ]:
# df.corr() for correlations
# correlation_matrix = df.corr()
# correlation_matrix['Spread']

In [ ]:
# load data
# create small subset to test code on
# get columns
# index by datetime
# add features - refer to feature eng file

In [2]:
df = pd.read_parquet('AAPL.parquet')

In [ ]:
df.corr()

In [ ]:
df.columns

In [ ]:


# ticker = ticker.partition("of ")[2].partition("to ")[0]
# features.loc[('BarDateTime', slice('08:30:00', '17:00:00'))]

In [3]:
datetime = df['BarDateTime'].dt.tz_localize(None)

In [4]:
# time = df['BarDateTime'].strptime("%d-%m-%y %H:%M:%S").second
hour = df['BarDateTime'].dt.hour
df['Hour'] = hour

In [5]:
del df['BarDateTime']
df['BarDateTime'] = datetime
# df['Time'] = time

In [ ]:
# date_generator_obj = df.sort_index().groupby('TradeDate','BarDateTime')

In [6]:
# df.sort_index().groupby('TradeDate', group_keys=False)
df.set_index(['TradeDate','BarDateTime'], inplace=True)

In [14]:
df.fillna(method='ffill')

Ticker  SecId OpenBarTimeOffset OpenBidPrice  \
TradeDate  BarDateTime                                                        
2017-12-01 2017-12-01 09:00:00   AAPL      0                 0       170.31   
           2017-12-01 09:01:00   AAPL      0                 0       170.35   
           2017-12-01 09:02:00   AAPL      0                 0       170.35   
           2017-12-01 09:03:00   AAPL      0                 0       170.35   
           2017-12-01 09:04:00   AAPL      0                 0       170.35   
...                               ...    ...               ...          ...   
2022-06-21 2022-06-21 15:55:00   AAPL  33449                 0       136.10   
           2022-06-21 15:56:00   AAPL  33449                 0       135.93   
           2022-06-21 15:57:00   AAPL  33449                 0       135.77   
           2022-06-21 15:58:00   AAPL  33449                 0       135.69   
           2022-06-21 15:59:00   AAPL  33449                 0       135.65   

                                OpenBidSize OpenAskPrice  OpenAskSize  \
TradeDate  BarDateTime                                                  
2017-12-01 2017-12-01 09:00:00          100       170.38          600   
           2017-12-01 09:01:00          300       170.38          600   
           2017-12-01 09:02:00          300       170.38          600   
           2017-12-01 09:03:00          300       170.38          600   
           2017-12-01 09:04:00          300       170.38          600   
...                                     ...          ...          ...   
2022-06-21 2022-06-21 15:55:00         2400       136.11          300   
           2022-06-21 15:56:00         4200       135.95         3900   
           2022-06-21 15:57:00         8800       135.78         8500   
           2022-06-21 15:58:00         3400       135.70         2400   
           2022-06-21 15:59:00         5600       135.66         3500   

                               FirstTradeTimeOffset FirstTradePrice  \
TradeDate  BarDateTime                                                
2017-12-01 2017-12-01 09:00:00          8.320261049        170.3800   
           2017-12-01 09:01:00          8.105317418        170.3800   
           2017-12-01 09:02:00          3.672846551        170.3500   
           2017-12-01 09:03:00          1.483471079        170.3800   
           2017-12-01 09:04:00          3.465247012        170.3800   
...                                             ...             ...   
2022-06-21 2022-06-21 15:55:00          0.004933554        136.1100   
           2022-06-21 15:56:00          0.001685085        135.9450   
           2022-06-21 15:57:00          0.042819026        135.7750   
           2022-06-21 15:58:00          0.000576060        135.6900   
           2022-06-21 15:59:00          0.000456978        135.6500   

                                FirstTradeSize  ... DowntickVolume  \
TradeDate  BarDateTime                          ...                  
2017-12-01 2017-12-01 09:00:00              75  ...            200   
           2017-12-01 09:01:00               1  ...             71   
           2017-12-01 09:02:00              10  ...             20   
           2017-12-01 09:03:00             187  ...            230   
           2017-12-01 09:04:00              18  ...              1   
...                                        ...  ...            ...   
2022-06-21 2022-06-21 15:55:00             400  ...          95550   
           2022-06-21 15:56:00             100  ...         102524   
           2022-06-21 15:57:00               1  ...          91015   
           2022-06-21 15:58:00             100  ...         111284   
           2022-06-21 15:59:00             100  ...         139456   

                               RepeatUptickVolume  RepeatDowntickVolume  \
TradeDate  BarDateTime                                                    
2017-12-01 2017-12-01 09:00:00                245                   

In [7]:
df = df[(df.Hour > 8) & (df.Hour < 16)]

In [ ]:
df.to_csv('AAPL.csv')

In [ ]:
# spread
# rel vol
# EMA/SMA close trade price
# EMA/SMA spread
# return
# volatility of price
# volatility of returns
# volatility of volume
# volatility of spread

##### Spread

In [29]:
# spread = df['CloseAskPrice'] - df['CloseBidPrice']
# spread
# del spread

df['Spread'] = df.apply(lambda x: x['CloseAskPrice'] - x['CloseBidPrice'], axis=1)

##### Returns

In [27]:
# returns = (df.apply(lambda x: ((x.LastTradePrice - x.FirstTradePrice)/x.FirstTradePrice)))
import decimal

# df['Returns'] = df.apply(lambda x: (x['LastTradePrice'] - x['FirstTradePrice'])/x['FirstTradePrice'], axis=1)

df['Returns'] = df.apply(lambda x:
     (x['LastTradePrice'] - x['FirstTradePrice'])/x['FirstTradePrice'] if x['FirstTradePrice'] != 0
     else decimal.Decimal('NaN'), axis=1)

##### Relative Volume

In [33]:
df['Relative_Volume_6'] = df.apply(lambda x: (df['Volume'])/(df['Volume'].rolling(6).mean()), axis=1)
df['Relative_Volume_12'] = df.apply(lambda x: (df['Volume'])/(df['Volume'].rolling(12).mean()), axis=1)
df['Relative_Volume_20'] = df.apply(lambda x: (df['Volume'])/(df['Volume'].rolling(20).mean()), axis=1)
df['Relative_Volume_30'] = df.apply(lambda x: (df['Volume'])/(df['Volume'].rolling(30).mean()), axis=1)
df['Relative_Volume_50'] = df.apply(lambda x: (df['Volume'])/(df['Volume'].rolling(50).mean()), axis=1)

# df['Relative_Volume_6'] = rel_vol_6
# df['Relative_Volume_12'] = rel_vol_12
# df['Relative_Volume_20'] = rel_vol_20
# df['Relative_Volume_30'] = rel_vol_30
# df['Relative_Volume_50'] = rel_vol_50

# del(rel_vol_6)
# del(rel_vol_12)
# del(rel_vol_20)
# del(rel_vol_30)
# del(rel_vol_50)

##### Relative Volatility Index (RVI)

In [ ]:
df['RVI'] = df.apply(lambda x: ta.rvi(high = x['HighTradePrice'],
                                      low = x['LowTradePrice'],
                                      close = x['LastTradePrice'],
                                      length = 14, refined = False,
                                      thirds = False, mamode = 'sma'), axis=1)

# df['RVI'] = rvi

# del(rvi)

##### Volatility of Closing Trade

In [ ]:
df['Volatility_StDev6_CloseTrade'] = (df.apply(lambda x: ta.stdev(length=6, close = x['LastTradePrice']), axis=1))
df['Volatility_StDev12_CloseTrade'] = (df.apply(lambda x: ta.stdev(length=12, close = x['LastTradePrice']), axis=1))
df['Volatility_StDev20_CloseTrade'] = (df.apply(lambda x: ta.stdev(length=20, close = x['LastTradePrice']), axis=1))
df['Volatility_StDev30_CloseTrade'] = (df.apply(lambda x: ta.stdev(length=30, close = x['LastTradePrice']), axis=1))
df['Volatility_StDev50_CloseTrade'] = (df.apply(lambda x: ta.stdev(length=50, close = x['LastTradePrice']), axis=1))

# df['Volatility_StDev6_CloseTrade'] = stdev_closetrade_6
# df['Volatility_StDev12_CloseTrade'] = stdev_closetrade_12
# df['Volatility_StDev20_CloseTrade'] = stdev_closetrade_20
# df['Volatility_StDev30_CloseTrade'] = stdev_closetrade_30
# df['Volatility_StDev50_CloseTrade'] = stdev_closetrade_50

# del(stdev_closetrade_6)
# del(stdev_closetrade_12)
# del(stdev_closetrade_20)
# del(stdev_closetrade_30)
# del(stdev_closetrade_50)

##### Volatility of Spread

In [ ]:
df['Volatility_StDev6_Spread'] = (df.apply(lambda x: ta.stdev(length=6, close = x['Spread']), axis=1))
df['Volatility_StDev12_Spread'] = (df.apply(lambda x: ta.stdev(length=12, close = x['Spread']), axis=1))
df['Volatility_StDev20_Spread'] = (df.apply(lambda x: ta.stdev(length=20, close = x['Spread']), axis=1))
df['Volatility_StDev30_Spread'] = (df.apply(lambda x: ta.stdev(length=30, close = x['Spread']), axis=1))
df['Volatility_StDev50_Spread'] = (df.apply(lambda x: ta.stdev(length=50, close = x['Spread']), axis=1))

# df['Volatility_StDev6_Spread'] = stdev_spread_6
# df['Volatility_StDev12_Spread'] = stdev_spread_12
# df['Volatility_StDev20_Spread'] = stdev_spread_20
# df['Volatility_StDev30_Spread'] = stdev_spread_30
# df['Volatility_StDev50_Spread'] = stdev_spread_50

# del(stdev_spread_6)
# del(stdev_spread_12)
# del(stdev_spread_20)
# del(stdev_spread_30)
# del(stdev_spread_50)

##### Volatility of Volume

In [ ]:
df['Volatility_StDev6_Volume'] = (df.apply(lambda x: ta.stdev(length=6, close = x['Volume']), axis=1))
df['Volatility_StDev12_Volume'] = (df.apply(lambda x: ta.stdev(length=12, close = x['Volume']), axis=1))
df['Volatility_StDev20_Volume'] = (df.apply(lambda x: ta.stdev(length=20, close = x['Volume']), axis=1))
df['Volatility_StDev30_Volume'] = (df.apply(lambda x: ta.stdev(length=30, close = x['Volume']), axis=1))
df['Volatility_StDev50_Volume'] = (df.apply(lambda x: ta.stdev(length=50, close = x['Volume']), axis=1))

# df['Volatility_StDev6_Volume'] = stdev_volume_6
# df['Volatility_StDev12_Volume'] = stdev_volume_12
# df['Volatility_StDev20_Volume'] = stdev_volume_20
# df['Volatility_StDev30_Volume'] = stdev_volume_30
# df['Volatility_StDev50_Volume'] = stdev_volume_50

# del(stdev_volume_6)
# del(stdev_volume_12)
# del(stdev_volume_12)
# del(stdev_volume_30)
# del(stdev_volume_50)

##### Volatility of Returns

In [ ]:
df['Volatility_StDev6_Returns'] = (df.apply(lambda x: ta.stdev(length=6, close = x['Returns']), axis=1))
df['Volatility_StDev12_Returns'] = (df.apply(lambda x: ta.stdev(length=12, close = x['Returns']), axis=1))
df['Volatility_StDev20_Returns'] = (df.apply(lambda x: ta.stdev(length=20, close = x['Returns']), axis=1))
df['Volatility_StDev30_Returns'] = (df.apply(lambda x: ta.stdev(length=30, close = x['Returns']), axis=1))
df['Volatility_StDev50_Returns'] = (df.apply(lambda x: ta.stdev(length=50, close = x['Returns']), axis=1))

# df['Volatility_StDev6_Returns'] = stdev_returns_6
# df['Volatility_StDev12_Returns'] = stdev_returns_12
# df['Volatility_StDev20_Returns'] = stdev_returns_20
# df['Volatility_StDev30_Returns'] = stdev_returns_30
# df['Volatility_StDev50_Returns'] = stdev_returns_50

# del(stdev_returns_6)
# del(stdev_returns_12)
# del(stdev_returns_12)
# del(stdev_returns_30)
# del(stdev_returns_50)

In [ ]:
# df.set_index('BarDateTime', inplace=True)
# grouped_df = df.groupby('TradeDate')

In [ ]:
# df2 = df.apply(lambda x: x.iloc[0:50])

In [ ]:
# for i in range(0, len(df), 24):
    # slc = df.iloc[i : i + 24]

In [ ]:
# by_ticker_date = df.sort_index().groupby(['BarDateTime'])
# remove_non_mkt_hrs = (by_ticker_date.apply(lambda x: [x['09:30:00':'16:00:00']]))

In [ ]:
# index
# remove premarket and after market data

In [ ]:
# calc spread and other features 

steps
1. drop features we know we won't need
2. add features
3. corr()
4. basic linear regression
5. try cross validation / LSTM / RNN